In [563]:
# !pip install soynlp
# !pip install gensim

In [1]:
import pandas as pd
import numpy as np
import re
from soynlp.tokenizer import RegexTokenizer
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus


tokenizer = RegexTokenizer()
tokenizer

In [2]:
# 곡 정보 파일 불러오기
df = pd.read_csv('data/song_tidy03.csv')

In [3]:
df.shape

(9582, 12)

In [4]:
# 가수 정보 파일 불러오기
df2 = pd.read_csv('data/artist_info_combined_ver04.csv', encoding='euc-kr')

In [5]:
# 곡 정보와 가수 정보를 머지해준다.
song = pd.merge(df, df2, on='artist')

In [6]:
# 가사 분석에 용이하도록 <br>을 공백으로 바꿔준다.
song['lyrics'] = song['lyrics'].str.replace(r'<br>', ' ')

In [214]:
whole_song = song['lyrics']
f = song[song['gender'] == '여']
whole_female = f['lyrics'].reset_index(drop=True)
m = song[song['gender'] == '남']
whole_male = m['lyrics'].reset_index(drop=True)

In [79]:
# 여자 전체곡수
female_len = whole_female.shape[0]
# 남자 전체곡수 
male_len = whole_male.shape[0]

In [80]:
# 타이틀 곡은 몇곡일까
len(song[song.is_title == '타이틀 곡'])

2379

## 해당 단어를 포함한 곡의 수 

In [86]:
female_k = whole_female[whole_female['lyrics'].str.contains("본능", na=False)]
female_k = int(female_k.shape[0])

In [87]:
male_k = whole_male[whole_male['lyrics'].str.contains("본능", na=False)]
male_k = int(male_k.shape[0])

In [90]:
# 여자그룹의 가수 곡 중에서 해당 단어의 등장 비율 보기 
(female_k/female_len)*100  # 0.4퍼 

0.4194128220491312

In [91]:
# 남자그룹의 가수 곡 중에서 해당 단어의 등장 비율 보기 
(male_k/male_len)*100  # 1퍼 

0.9712046345203612

## 단어 전처리 및 토크나이징

In [16]:
def preprocessing(text):
    # 개행문자 제거
    text =  text.strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [17]:
def tokenize(df):
    sentences = df['lyrics'].apply(preprocessing)
    tokens = sentences.apply(tokenizer.tokenize)
    tokens = tokens.tolist()
    tokens_all = sum(tokens, [])
    return tokens_all

In [18]:
# 토크나이징 적용

song_t = tokenize(song)  # 토크나이징 된 것이 해당 song_t로 들어간다.
female_t = tokenize(female)
male_t = tokenize(male)

# 하다가 꼬였는가? copy 한거 쓰자!!!!!

In [19]:
# copy()

song_t = song_t.copy()
female_t = female_t.copy()
male_t = male_t.copy()

In [20]:
len(female_t)  # 여그룹 총 토크나이징된 단어 769,211  약 77만 

769211

In [21]:
len(male_t)  # 남그룹 총 토크나이징된 단어  1,409,422  약 141만

1409422

In [698]:
# male_t  # 하나로 합쳐져있다.

## 앞/뒤에 오는 단어를 보는 함수

In [22]:
# 중복되는 구절이 있어서 만든 유니크한 값만 보는 함수

def unique(word):
    result = []
    for i in word:
        if i not in result: #inside for
            result.append(i) #inside for inside if
    return result
    print(result)

In [270]:
# 앞 5글자, 뒤 5글자 한번에 보는 함수 
    
def n_f_word(tokenized_list, search_word):
    indices2 = [i for i, x in enumerate(tokenized_list) if x == search_word]
    i = 0
    result = []
    for i in range(len(indices2)):
        next_word_2 = tokenized_list[indices2[i] - 5 :indices2[i] + 6]  #
        result.append(next_word_2[1:])
        i+=1
    return result


In [284]:
# 두 음절 이상의 단어중에서 빈도수 상위 30 번째까지 출력 
from collections import Counter

topn = 30 

def more_word(word):     
    more_words = []
    for v1 in word:
        for v2 in v1:
             if len(v2) > 1:
                more_words.append(v2)
    result = Counter(more_words).most_common(topn)
    return result

# 원하는 키워드가 들어간 문장 '한 노래당 하나씩만' 보기 

In [147]:
# EX) '잘생' 이라는 글자가 등장하면 곡 당 제일 먼저 걸리는 한 구절만 가져오게 했습니다.

In [176]:
len(lyrics)

9566

In [215]:
len(whole_male)

5869

In [ ]:
넥타이,수트(정장) --> (비교) 

3. 신체부위 : 단어 빈도 = > 맥락확인 (Keyword in context 픽토그램)
-얼굴, 눈,코,입(술),
-허리, 다리, 엉덩이
-몸매 
-근육, 어깨 (남자?), 등(남자?) ,키
-화장
(수치적으로)


In [266]:
lyrics = whole_song  # whole_song(전체곡 중에서 보기), whole_female(여성그룹 중에서 보기), whole_male(남성그룹 중에서 보기)
result = []

import re

for i in range(len(lyrics)):
    texts = lyrics[i]
    f = re.search(r'(.{0,40}(Ss)uit.{0,40})', texts)
    if f is not None:
        result.append('노래번호 (' + str(i) + ') ' + f[0])

In [267]:
len(result) # 등장한 곡 수 

0

In [268]:
result  # EX) '잘생' 이라는 글자가 등장하면 곡 당 제일 먼저 걸리는 한 구절만 가져오게 했습니다.

[]

In [226]:
# lyrics[315]  # 전체 가사로 확인해보기 

#  맥락 속 등장단어의 비율보기

In [362]:
def excute_both_both(word):

    female_both_words = more_word(unique(n_f_word(female_t, word)))  
    male_both_words = more_word(unique(n_f_word(male_t, word)))
    
    female_both_len = len(n_f_word(female_t, word))
    male_both_len = len(n_f_word(male_t, word))
    
    return (both_words_ratio(female_both_words, male_both_words, female_both_len, male_both_len))
   


In [369]:
def both_words_ratio(f, m, f_count, m_count):
    
    f = pd.DataFrame(f)
    m = pd.DataFrame(m)
    f.columns = ['words', 'f_count/girl_group']
    m.columns = ['words', 'm_count/boy_group']

    merged_ = pd.merge(f, m, how='outer', on=['words'])    
    f['count / girl_group'] = (merged_['f_count/girl_group']/female_len)*100
    m['count / boy_group'] = (merged_['m_count/boy_group']/male_len)*100

    f = f.dropna()
    m = m.dropna()
    
    return (pd.merge(f, m, how='outer', on=['words']))


In [374]:
result_ratio = excute_both_both('엉덩이')

In [375]:
result_ratio.head()

,words,f_count/girl_group,count / girl_group,m_count/boy_group,count / boy_group
0,엉덩이,13.0,0.389455,3.0,0.051116
1,it,7.0,0.209706,NaN,NaN
2,shake,6.0,0.179748,NaN,NaN
3,머리에는,5.0,0.149790,NaN,NaN
4,다리에는,5.0,0.149790,NaN,NaN


In [376]:
result_ratio.to_excel('nlp_result/엉덩이.xlsx')